In [1]:
import os
import plotly 

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


data_dir = 'data'

df = pd.read_csv(os.path.join(data_dir, '2023-03-13', 'ABRD_2023-03-13_20-37-28.csv'))
df['TRADETIME'] = pd.to_datetime(df['TRADETIME'])

df = df.sort_values('TRADETIME')

features = ['TRADETIME', 'PRICE', 'QUANTITY', 'VALUE', 'BUYSELL']


C:\Users\igorl\AppData\Local\Temp\ipykernel_24360\1835525300.py:13: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TRADETIME'] = pd.to_datetime(df['TRADETIME'])


In [2]:
# ewms = []
# for side in ['B', 'S']:
#     sub_df = df.loc[df['BUYSELL'] == 'B']
#     ewms.append(sub_df.PRICE.ewm(
#         halflife='360 seconds', times=sub_df['TRADETIME']).mean())

# df['mean_price'] = np.where(df['BUYSELL'] == 'B', ewms[0], ewms[1])


In [3]:
# add ewm mean and std over 6 mins 
df['mean_price_b'] = df.loc[df['BUYSELL'] == 'B'].PRICE.ewm(
    halflife='360 seconds', times=df.loc[df['BUYSELL'] == 'B', 'TRADETIME']).mean()
df['std_price_b'] = df.loc[df['BUYSELL'] == 'B'].PRICE.ewm(
    halflife='360 seconds', times=df.loc[df['BUYSELL'] == 'B', 'TRADETIME']).std()
df['mean_quantity_b'] = df.loc[df['BUYSELL'] == 'B'].QUANTITY.ewm(
    halflife='360 seconds', times=df.loc[df['BUYSELL'] == 'B', 'TRADETIME']).mean()
df['std_quantity_b'] = df.loc[df['BUYSELL'] == 'B'].QUANTITY.ewm(
    halflife='360 seconds', times=df.loc[df['BUYSELL'] == 'B', 'TRADETIME']).std()

# add ewm mean and std over 6 mins
df['mean_price_s'] = df.loc[df['BUYSELL'] == 'S'].PRICE.ewm(
    halflife='360 seconds', times=df.loc[df['BUYSELL'] == 'S', 'TRADETIME']).mean()
df['std_price_s'] = df.loc[df['BUYSELL'] == 'S'].PRICE.ewm(
    halflife='360 seconds', times=df.loc[df['BUYSELL'] == 'S', 'TRADETIME']).std()
df['mean_quantity_s'] = df.loc[df['BUYSELL'] == 'S'].QUANTITY.ewm(
    halflife='360 seconds', times=df.loc[df['BUYSELL'] == 'S', 'TRADETIME']).mean()
df['std_quantity_s'] = df.loc[df['BUYSELL'] == 'S'].QUANTITY.ewm(
    halflife='360 seconds', times=df.loc[df['BUYSELL'] == 'S', 'TRADETIME']).std()


In [6]:
quantity_anomalies = df[df.QUANTITY > df.mean_quantity + 2 * df.std_quantity]
price_anomalies = df[np.sqrt((df.PRICE - df.mean_price)**2) / df.PRICE > .05]

AttributeError: 'DataFrame' object has no attribute 'mean_price'

In [37]:
import plotly.express as px

fig = px.line(df, x='TRADETIME', y='PRICE', color='BUYSELL')
# fig = px.line(df, x='TRADETIME', y='VALUE')


# fig.add_vline(x=0.9)

for index, a in quantity_anomalies.iterrows():
    fig.add_vline(x=a.TRADETIME)

for index, a in price_anomalies.iterrows():
    fig.add_vline(x=a.TRADETIME)


fig.show()


In [ ]:
def check_dummy_anomalies(df):
    df